<a href="https://colab.research.google.com/github/aayushrai/Mask_Recognition/blob/master/Train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
!unzip "/content/drive/My Drive/type_of_mask_dataset.zip"

In [13]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten,Conv2D,MaxPooling2D,Activation
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

In [38]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        'dataset',
        target_size=(150, 150),
        batch_size=32,color_mode="grayscale")
validation_generator = test_datagen.flow_from_directory(
        'dataset',
        target_size=(150, 150),
        batch_size=32,color_mode="grayscale")

Found 2669 images belonging to 4 classes.
Found 2669 images belonging to 4 classes.


In [39]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation('sigmoid'))

In [40]:
INIT_LR = 1e-4
EPOCHS = 20
BS = 32

print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])

[INFO] compiling model...


In [41]:
# train the head of the network
print("[INFO] training head...")
H = model.fit(
	train_generator,
	steps_per_epoch=124,
	validation_data=validation_generator,
	validation_steps=124,epochs=EPOCHS)

[INFO] training head...
Epoch 1/20
124/124 [==============================] - 143s 1s/step - loss: 1.3447 - accuracy: 0.3317 - val_loss: 1.2237 - val_accuracy: 0.4814
Epoch 2/20
124/124 [==============================] - 89s 721ms/step - loss: 1.1927 - accuracy: 0.4576 - val_loss: 1.1105 - val_accuracy: 0.5503
Epoch 3/20
124/124 [==============================] - 90s 727ms/step - loss: 1.1034 - accuracy: 0.5074 - val_loss: 0.9823 - val_accuracy: 0.6481
Epoch 4/20
124/124 [==============================] - 90s 725ms/step - loss: 1.0200 - accuracy: 0.5634 - val_loss: 0.9194 - val_accuracy: 0.6169
Epoch 5/20
124/124 [==============================] - 89s 715ms/step - loss: 0.9614 - accuracy: 0.5964 - val_loss: 0.8360 - val_accuracy: 0.6527
Epoch 6/20
124/124 [==============================] - 90s 728ms/step - loss: 0.9261 - accuracy: 0.6161 - val_loss: 0.7871 - val_accuracy: 0.6969
Epoch 7/20
124/124 [==============================] - 89s 714ms/step - loss: 0.8699 - accuracy: 0.6356 - val

In [20]:
print("[INFO] saving mask detector model...")
model.save("mask_type.h5", save_format="h5")

[INFO] saving mask detector model...
